In [1]:
import sys
import os
import time
import h5py
import numpy as np
sys.path.append("..")  # Adds higher directory to python modules path.
from img_to_vec import Img2Vec
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC


#import torchvision.models as models
# model = models.resnet152(pretrained=True)
# modules=list(model.modules())[:-1]
# print(*modules)

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests 

In [2]:

# input_path = './test_images'
input_path = '/home/mikaelnb/Project/notebooks/YouTubeFacesDB/data'
#input_path = './faces/'

img2vec = Img2Vec(model='resnet-152')

os.listdir(input_path)

/home/mikaelnb/.local/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


['frame_images_DB',
 'sources',
 'headpose_DB',
 'meta_data',
 'aligned_images_DB',
 'images.txt',
 'descriptors_DB']

In [3]:
aligned_folder = '/aligned_images_DB/'
new_input_path = input_path + aligned_folder
labels = sorted(os.listdir(new_input_path), key=lambda s: s.lower())[:100]
label_id = {}
for i, label in enumerate(labels):
    label_id[i] = label
print(labels, label_id)

['Aaron_Eckhart', 'Aaron_Guiel', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abba_Eban', 'Abbas_Kiarostami', 'Abdel_Aziz_Al-Hakim', 'Abdoulaye_Wade', 'Abdul_Majeed_Shobokshi', 'Abdul_Rahman', 'Abdulaziz_Kamilov', 'Abdullah', 'Abdullah_Ahmad_Badawi', 'Abdullah_Gul', 'Abdullatif_Sener', 'Abel_Pacheco', 'Abid_Hamid_Mahmud_Al-Tikriti', 'Abraham_Foxman', 'Adam_Ant', 'Adam_Sandler', 'Adam_Scott', 'Adel_Al-Jubeir', 'Adisai_Bodharamik', 'Adoor_Gopalakarishnan', 'Adrian_Annus', 'Adrian_Fernandez', 'Adrian_Nastase', 'Adriana_Lima', 'Adriana_Perez_Navarro', 'Adrianna_Zuzic', 'Adrien_Brody', 'Agbani_Darego', 'Agnes_Bruckner', 'Ahmad_Masood', 'Ahmed_Ahmed', 'Ahmed_Chalabi', 'Ahmed_Qureia', 'Ahmet_Necdet_Sezer', 'Aidan_Quinn', 'Aishwarya_Rai', 'AJ_Cook', 'Akbar_Al_Baker', 'Akbar_Hashemi_Rafsanjani', 'Akhmed_Zakayev', 'Akiko_Morigami', 'Akmal_Taher', 'Al_Davis', 'Al_Gore', 'Al_Leiter', 'Al_Pacino', 'Al_Sharpton', 'Alain_Ducasse', 'Alan_Ball', 'Alan_Dershowitz', 'Alanis_Morissette', 'Alanna_Ubach', 'Alastair_Cam

In [4]:
# for file in os.listdir(new_input_path + label):
#     print(os.fsdecode(file))

In [6]:
print(len(label_id))
#pics = np.empty((len(label_id),0))
pics = [None]*len(label_id)
#print(pics)

for index in label_id:
    pics[int(index)] = []
    
#print(pics)


100
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

In [7]:
vec = []
labels = []
imagesPerLabel = 10

testVec = []
k = 0

for index in label_id:    
    for root, dirs, files in os.walk(new_input_path + label_id[index]):
        #print(root, dirs, files)
        for name in files:
            if len(pics[index]) > imagesPerLabel -1:
                print(index, len(pics[index]))
                break
            #print(name)
            if name.endswith((".jpg")):                
                img = Image.open(os.path.join(os.fsdecode(root), name))
                #print(os.path.join(os.fsdecode(root), name))
                vec = img2vec.get_vec(img)
                if  testVec == [] and index == 0:
                    k += 1
                    if k == 5:
                        print('index: ', index)
                        testVec = vec
                print(vec.shape)                
                pics[index].append(vec)     
                #print(name)
                labels.append(int(index))
    pics[index] = np.array(pics[index])    
    
labels = np.array(labels)

# print()
# print(pics[0][0:])
# print(pics[1][0:])

#                 X = vec.T
#                 num_components = 128
#                 pca = PCA(n_components = num_components, whiten=True).fit(X)

#                 X_train = pca.transform(X)

(2048,)
(2048,)
(2048,)
(2048,)
index:  0
(2048,)


/home/mikaelnb/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
0 10
0 10
0 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
1 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
2 10
2 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
3 10
3 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
4 10
4 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
5 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
6 10
6 10
6 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
7 10
7 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
8 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
9 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
10 10
10 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2

(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
89 10
89 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
90 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
91 10
91 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
92 10
92 10
92 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
93 10
93 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
94 10
94 10
94 10
94 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
95 10
95 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
96 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
97 10
97 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
98 10
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
(2048,)
99 10
99 10
99 10
99 10

[[0.12123615 0.

In [8]:
pics=np.array(pics)
pics = pics.reshape(len(pics)*len(pics[0]),2048)
print(pics.shape)

print(labels)

(1000, 2048)
[ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2
  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4
  4  4  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  7  7
  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 14 14 14 14
 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16
 16 16 17 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 19 19
 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20 20 20 21 21 21 21 21 21
 21 21 21 21 22 22 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 23 23 23
 24 24 24 24 24 24 24 24 24 24 25 25 25 25 25 25 25 25 25 25 26 26 26 26
 26 26 26 26 26 26 27 27 27 27 27 27 27 27 27 27 28 28 28 28 28 28 28 28
 28 28 29 29 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 30 30 31 31
 31 31 31 31 31 31 31 31 32 32 32 32 3

In [9]:
# import random
# n=25
# randList=[]

# for i in range(0,len(pics)):
#     randList.append(random.sample(range(imagesPerLabel), n))
# randList

In [10]:
# newPics = [None]*len(label_id)
# for index in label_id:
#     newPics[int(index)] = []
    
# for index in range(0,len(pics)):
#     for i in range(0, n):
#         newPics[index].append(pics[index][randList[index][i]])
#     newPics[index] = np.array(newPics[index])
# newPics = np.array(newPics)

In [11]:
X = pics
y = labels

In [12]:

#from sklearn.preprocessing import LabelEncoder

with h5py.File("./YTF_hdf5/data_ytf.hdf5", "w") as f:
    dset = f.create_dataset("dataset_1", data=np.array(X[:100]))

#le = LabelEncoder()
#target = le.fit_transform(labels)
with h5py.File("./YTF_hdf5/labels_ytf.hdf5", "w") as f:
    dset = f.create_dataset("dataset_1", data=np.array(y[:100]))
print(X.shape, y.shape)

(1000, 2048) (1000,)


In [13]:
f1 = h5py.File('./YTF_hdf5/data_ytf.hdf5', 'r')
X = f1.get('dataset_1').value # `data` is now an ndarray.
f1.close()
#X = np.array(X)

f2 = h5py.File('./YTF_hdf5/labels_ytf.hdf5', 'r')
y = f2.get('dataset_1').value # `data` is now an ndarray.
f2.close()
#y = np.array(y)
print(X.shape, y.shape)

(100, 2048) (100,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state = 6)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
#print(X.shape, y.shape)

(67, 2048) (67,) (33, 2048) (33,)


In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.9696969696969697

In [16]:
scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0


In [17]:
# search for an optimal value of K for KNN

# range of k we want to try
k_range = range(1, 19)
# empty list to store scores
k_scores = []

# 1. we will loop through reasonable values of k
for k in k_range:
    # 2. run KNeighborsClassifier with k neighbours
    knn = KNeighborsClassifier(n_neighbors=k)
    # 3. obtain cross_val_score for KNeighborsClassifier with k neighbours
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    # 4. append mean of scores for k neighbors to k_scores list
    k_scores.append(scores.mean())


print(k_scores)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.9800000000000001, 0.97, 0.96, 0.95, 0.9400000000000001, 0.9200000000000002, 0.9100000000000001, 0.9, 0.9, 0.89, 0.86]


In [18]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

#testVec = [x/n for x in [sum(x) for x in newPics]]
#print(neigh.predict([testVec]))
#print(neigh.predict_proba([testVec]))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [19]:
start_time = time.time()
clf = KNeighborsClassifier(n_neighbors=1, p=1)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0
56.47110939025879 ms


In [20]:
start_time = time.time()
clf = DecisionTreeClassifier(max_depth=250, min_samples_split=2)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[0.8 0.9 1.  1.  0.7 0.8 1.  0.9 0.8 1. ]
0.8899999999999999
727.0896434783936 ms


In [21]:
start_time = time.time()
clf = RandomForestClassifier(max_depth=250, n_estimators=200, max_features=5)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0
2644.1617012023926 ms


In [22]:
start_time = time.time()
clf = MLPClassifier(alpha=3.1)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))


/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0
17433.52961540222 ms


/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# AdaboostClassifier

In [23]:
start_time = time.time()
clf = AdaBoostClassifier(n_estimators=100, learning_rate = 0.5)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[0.9 1.  0.9 1.  0.9 1.  1.  0.9 0.9 0.9]
0.9400000000000001
18336.39931678772 ms


# Bayes

In [24]:
start_time = time.time()
clf = GaussianNB()
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[1.  1.  1.  1.  0.9 0.9 1.  0.9 1.  1. ]
0.97
52.93774604797363 ms


In [25]:
start_time = time.time()
clf = QuadraticDiscriminantAnalysis()
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())


print("%s ms" % ((time.time() - start_time)*1000))

[0.7 0.5 0.5 0.4 0.5 0.3 0.7 0.4 0.3 0.3]
0.45999999999999996
172.4092960357666 ms


/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mikaelnb/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are coll

In [26]:
start_time = time.time()
clf = SVC(gamma=1, C=1)
clf.fit(X_train,y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

print("%s ms" % ((time.time() - start_time)*1000))

[0.3 0.4 0.3 0.8 0.3 0.4 0.2 0.2 0.3 0.2]
0.34
345.8590507507324 ms
